In [2]:
import csv
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 1. Read data

In [3]:
# Taiwan_201701 = pd.read_csv('sinica/201701_Taiwan.csv')
# Taiwan_201702 = pd.read_csv('sinica/201702_Taiwan.csv')
# Taiwan_201703 = pd.read_csv('sinica/201703_Taiwan.csv')

# Taiwan_201701 = Taiwan_201701.rename(columns={' lat': 'lat', ' lon': 'lon'})
# Taiwan_201702 = Taiwan_201702.rename(columns={' lat': 'lat', ' lon': 'lon'})

# epa_loc_data_201701 = pd.read_csv('epa/EPA_LOC_data_201701.csv')

In [4]:
Taiwan_201701_loc = pd.read_csv('Taiwan_201701_loc.csv')
Taiwan_201702_loc = pd.read_csv('Taiwan_201702_loc.csv')
Taiwan_201703_loc = pd.read_csv('Taiwan_201703_loc.csv')

In [5]:
Taiwan_201701_loc[:5]

,Date,Time,device_id,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,loc
0,2017-01-01,08:00:00,74DA388FF60A,31.0,33.0,22.0,22.75,78.0,25.072,121.657,新北市汐止區
1,2017-01-01,08:00:00,74DA3895DF64,59.0,76.0,40.0,21.25,92.0,22.963,120.325,高雄市楠梓區
2,2017-01-01,08:00:00,74DA388FF60A,31.0,33.0,22.0,22.75,78.0,25.072,121.657,新北市汐止區
3,2017-01-01,08:00:00,74DA3895DF64,59.0,76.0,40.0,21.25,92.0,22.963,120.325,高雄市楠梓區
4,2017-01-01,08:00:00,74DA388FF60A,31.0,33.0,22.0,22.75,78.0,25.072,121.657,新北市汐止區


## 2. Preprocessing

In [6]:
def drop_outliers(df):
    
    def _drop_PM10(df):
        PM10_CI95p = df.PM10.mean() + 2 * df.PM10.std()
        PM10_CI95n = df.PM10.mean() - 2 * df.PM10.std()
        df = df.drop(df[df.PM10 > PM10_CI95p].index)
        df = df.drop(df[df.PM10 < PM10_CI95n].index)
        df = df.drop(df[df.PM10 == 0].index)
        return df

    def _drop_PM1(df):
        PM1_CI95p = df.PM1.mean() + 2 * df.PM1.std()
        PM1_CI95n = df.PM1.mean() - 2 * df.PM1.std()
        df = df.drop(df[df.PM1 > PM1_CI95p].index)
        df = df.drop(df[df.PM1 < PM1_CI95n].index)
        df = df.drop(df[df.PM1 == 0].index)
        return df
    
    def _drop_temperature(df):
        temperature_CI95p = df.Temperature.mean() + 2 * df.Temperature.std()
        temperature_CI95n = df.Temperature.mean() - 2 * df.Temperature.std()
        df = df.drop(df[df.Temperature > temperature_CI95p].index)
        df = df.drop(df[df.Temperature < temperature_CI95n].index)
        return df
    
    def _drop_humidity(df):
        humidity_CI95p = df.Humidity.mean() + 2 * df.Humidity.std()
        humidity_CI95n = df.Humidity.mean() - 2 * df.Humidity.std()
        df = df.drop(df[df.Humidity > humidity_CI95p].index)
        df = df.drop(df[df.Humidity < humidity_CI95n].index)
        return df
    
    before = df.shape
    
    df = _drop_PM10(df)
    df = _drop_PM1(df)
    df = _drop_temperature(df)
    df = _drop_humidity(df)
    
    after = df.shape
    
    print(before, ' -> ', after)
    
    return df

In [7]:
# Taiwan_201701_CI95 = drop_outliers(Taiwan_201701_loc)
# Taiwan_201702_CI95 = drop_outliers(Taiwan_201702_loc)
# Taiwan_201703_CI95 = drop_outliers(Taiwan_201703_loc)

In [8]:
def preprocessing(df):
    
    def _normalization(df):
        for feature_name in ['PM10', 'PM1', 'Temperature', 'Humidity']:
            max_value = df[feature_name].mean() + 2 * df[feature_name].std()
            min_value = df[feature_name].mean() - 2 * df[feature_name].std()
            df[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
        print('normalization DONE!')
        return df
    
    def _concat_datetime(df):
        df['period'] = df[['Date', 'Time']].apply(lambda x: ' '.join(x), axis=1)
        df = df.drop(['Date', 'Time'], axis=1)
        print('concat_datetime DONE!')
        return df
    
    def _cluster_loc(df):
        global epa_loc_data_201701
        counter = 0
        loc_list = []
        display_steps = int(len(df) / 100)
        for lon, lat in zip(df['lon'], df['lat']):
            min_distance = 999.9
            for row in epa_loc_data_201701.itertuples():
                loc_name = row[1]
                loc_lon = row[2]
                loc_lat = row[3]
                distance = (lon-loc_lon) ** 2 + (lat-lat) ** 2
                if min_distance > distance:
                    min_distance = distance
                    best_loc = loc_name
            loc_list.append(best_loc)
            if counter % display_steps == 0:
                p = int(counter / display_steps)
                print('[%s] %d/100' % (time.strftime("%H:%M:%S", time.localtime()), p))
            counter += 1
        df['loc'] = pd.Series(loc_list).values
        print('cluster_loc DONE!')
        return df
    
    def _drop_redundant_features(df):
        df = df.drop(['device_id', 'lat', 'lon'], axis=1)
        print('drop_redundant_features DONE!')
        return df
    
    df = _normalization(df)
    df = _concat_datetime(df)
#     df = _cluster_loc(df)
    df = _drop_redundant_features(df)
    return df

In [9]:
Taiwan_201701_precd = preprocessing(Taiwan_201701_loc)
Taiwan_201702_precd = preprocessing(Taiwan_201702_loc)
Taiwan_201703_precd = preprocessing(Taiwan_201703_loc)

normalization DONE!
concat_datetime DONE!
drop_redundant_features DONE!
normalization DONE!
concat_datetime DONE!
drop_redundant_features DONE!
normalization DONE!
concat_datetime DONE!
drop_redundant_features DONE!


## 3. Grouping

In [10]:
Taiwan_201701_group = Taiwan_201701_precd.groupby('loc') 
Taiwan_201702_group = Taiwan_201702_precd.groupby('loc')
Taiwan_201703_group = Taiwan_201703_precd.groupby('loc')

In [11]:
def get_largest_group(df):
    max_name = ''
    max_len = 0
    for name, group in df:
        length = len(group)
        if max_len < length:
            max_name = name
            max_len = length
    print(max_name, max_len)

In [13]:
train_group = Taiwan_201701_group.get_group('臺中市南屯區')
valid_group = Taiwan_201702_group.get_group('臺南市中西區')

## 4. Preprocessing of groups

In [14]:
def preprocessing_datetime_group(df):
    
    def _convert_datetime_to_norm_sec(df):
        df['period'] = pd.to_datetime(df['period'])
        df['seconds'] = [time.mktime(t.timetuple()) for t in df.period]
        max_value = df['seconds'].max()
        min_value = df['seconds'].min()
        df['seconds'] = (df['seconds'] - min_value) / (max_value - min_value)
        return df
    
    def _drop_redundant_features(df):
        X = df.drop(['loc', 'period', 'PM2.5'], axis=1).values
        Y = df['PM2.5'].values
        return X, Y
    
    def _reshapeX(X):
        X = X.reshape(X.shape[0], 1, X.shape[-1])
        return X
        
#     df = _convert_datetime_to_norm_sec(df)
    X, Y = _drop_redundant_features(df)
    X = _reshapeX(X)
    
    return X, Y

In [15]:
trainX, trainY = preprocessing_datetime_group(train_group)
validX, validY = preprocessing_datetime_group(valid_group)

In [16]:
print(trainX.shape, trainY.shape, validX.shape, validY.shape)

(299591, 1, 4) (299591,) (107365, 1, 4) (107365,)


## 5. Build model

In [17]:
def LSTM_PM25(trainX, trainY, validX, validY, output_dim, epoch, batch_size):
    model = Sequential()
    model.add(LSTM(output_dim, input_shape=(trainX.shape[1], trainX.shape[2])))    
    model.add(Dense(units=1, kernel_initializer='uniform', activation='relu'))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(trainX, trainY, epochs=epoch, batch_size=batch_size, validation_data=(validX, validY), verbose=1, shuffle=False)
    return model

In [18]:
model = LSTM_PM25(trainX, trainY, validX, validY, 50, 10, 100)

Train on 299591 samples, validate on 107365 samples
Epoch 1/10
299591/299591 [==============================] - 28s 95us/step - loss: 562.3330 - val_loss: 422.0652
Epoch 2/10
299591/299591 [==============================] - 26s 88us/step - loss: 118.2121 - val_loss: 56.6789
Epoch 3/10
299591/299591 [==============================] - 26s 88us/step - loss: 77.3600 - val_loss: 24.0811
Epoch 4/10
299591/299591 [==============================] - 26s 88us/step - loss: 64.0236 - val_loss: 18.5980
Epoch 5/10
299591/299591 [==============================] - 26s 88us/step - loss: 57.2954 - val_loss: 17.9604
Epoch 6/10
299591/299591 [==============================] - 26s 88us/step - loss: 53.2062 - val_loss: 17.4954
Epoch 7/10
299591/299591 [==============================] - 26s 88us/step - loss: 50.0336 - val_loss: 16.9485
Epoch 8/10
299591/299591 [==============================] - 26s 88us/step - loss: 47.2286 - val_loss: 15.8553
Epoch 9/10
299591/299591 [==============================] - 26s 8

In [25]:
model15 = LSTM_PM25(trainX, trainY, validX, validY, 50, 15, 100)
model20 = LSTM_PM25(trainX, trainY, validX, validY, 50, 20, 100)
model30 = LSTM_PM25(trainX, trainY, validX, validY, 50, 30, 100)

Train on 299591 samples, validate on 107365 samples
Epoch 1/15
299591/299591 [==============================] - 27s 90us/step - loss: 561.9239 - val_loss: 415.0429
Epoch 2/15
299591/299591 [==============================] - 26s 88us/step - loss: 117.7199 - val_loss: 55.7910
Epoch 3/15
299591/299591 [==============================] - 26s 88us/step - loss: 77.6894 - val_loss: 24.0224
Epoch 4/15
299591/299591 [==============================] - 26s 88us/step - loss: 64.4636 - val_loss: 18.1972
Epoch 5/15
299591/299591 [==============================] - 26s 88us/step - loss: 57.3812 - val_loss: 16.9865
Epoch 6/15
299591/299591 [==============================] - 26s 88us/step - loss: 52.9830 - val_loss: 15.6727
Epoch 7/15
299591/299591 [==============================] - 26s 88us/step - loss: 49.5803 - val_loss: 15.1722
Epoch 8/15
299591/299591 [==============================] - 26s 88us/step - loss: 46.6668 - val_loss: 14.8129
Epoch 9/15
299591/299591 [==============================] - 26s 8

In [30]:
model40 = LSTM_PM25(trainX, trainY, validX, validY, 50, 40, 100)
model50 = LSTM_PM25(trainX, trainY, validX, validY, 50, 50, 100)
model60 = LSTM_PM25(trainX, trainY, validX, validY, 50, 60, 100)

Train on 299591 samples, validate on 107365 samples
Epoch 1/40
299591/299591 [==============================] - 28s 92us/step - loss: 561.0158 - val_loss: 418.0592
Epoch 2/40
299591/299591 [==============================] - 27s 90us/step - loss: 117.6552 - val_loss: 56.4108
Epoch 3/40
299591/299591 [==============================] - 27s 90us/step - loss: 77.3824 - val_loss: 24.0728
Epoch 4/40
299591/299591 [==============================] - 27s 90us/step - loss: 64.1663 - val_loss: 18.4562
Epoch 5/40
299591/299591 [==============================] - 27s 90us/step - loss: 57.3455 - val_loss: 17.3821
Epoch 6/40
299591/299591 [==============================] - 27s 90us/step - loss: 53.0863 - val_loss: 15.7423
Epoch 7/40
299591/299591 [==============================] - 27s 90us/step - loss: 49.7020 - val_loss: 15.7708
Epoch 8/40
299591/299591 [==============================] - 27s 90us/step - loss: 46.7832 - val_loss: 15.7954
Epoch 9/40
299591/299591 [==============================] - 27s 9

Epoch 34/50
299591/299591 [==============================] - 27s 89us/step - loss: 13.0820 - val_loss: 13.0024
Epoch 35/50
299591/299591 [==============================] - 27s 89us/step - loss: 12.5576 - val_loss: 13.1040
Epoch 36/50
299591/299591 [==============================] - 27s 89us/step - loss: 12.0610 - val_loss: 13.2496
Epoch 37/50
299591/299591 [==============================] - 27s 90us/step - loss: 11.5840 - val_loss: 13.4379
Epoch 38/50
299591/299591 [==============================] - 27s 89us/step - loss: 11.1412 - val_loss: 13.5527
Epoch 39/50
299591/299591 [==============================] - 27s 89us/step - loss: 10.7317 - val_loss: 13.6076
Epoch 40/50
299591/299591 [==============================] - 27s 89us/step - loss: 10.3464 - val_loss: 13.6231
Epoch 41/50
299591/299591 [==============================] - 27s 90us/step - loss: 9.9895 - val_loss: 13.6157
Epoch 42/50
299591/299591 [==============================] - 27s 89us/step - loss: 9.6623 - val_loss: 13.6249
Epo

299591/299591 [==============================] - 27s 89us/step - loss: 6.9221 - val_loss: 14.7258
Epoch 58/60
299591/299591 [==============================] - 27s 89us/step - loss: 6.7734 - val_loss: 14.9500
Epoch 59/60
299591/299591 [==============================] - 27s 89us/step - loss: 6.6371 - val_loss: 15.3009
Epoch 60/60
299591/299591 [==============================] - 27s 89us/step - loss: 6.5107 - val_loss: 15.5506


## 6. Predict 201703 PM2.5

In [31]:
def predict_201703_PM25(model):
    result = [None] * len(Taiwan_201703_precd)

    counter = 0
    total = len(Taiwan_201703_group)

    for name, group in Taiwan_201703_group:
        index_list = group.index.tolist()
        testX = group.drop(['PM2.5', 'loc', 'period'], axis=1)
        testX = testX.values
        testX = testX.reshape(testX.shape[0], 1, testX.shape[-1])
        testY_hat_list = model.predict(testX).tolist()

        for index, Y_hat in zip(index_list, testY_hat_list):
            result[index] = Y_hat[0]

        counter += 1
        print('%s Finished! %d/%d' % (name, counter, total))
        
        result_df = pd.DataFrame(result, columns=["PM2.5"])
        result_df = result_df.round(0)
        
    return result_df

In [28]:
result15 = predict_201703_PM25(model15)
result20 = predict_201703_PM25(model20)
result30 = predict_201703_PM25(model30)

南投縣南投市 Finished! 1/70
南投縣埔里鎮 Finished! 2/70
南投縣竹山鎮 Finished! 3/70
嘉義市西區 Finished! 4/70
嘉義縣新港鄉 Finished! 5/70
嘉義縣朴子市 Finished! 6/70
基隆市信義區 Finished! 7/70
宜蘭縣冬山鄉 Finished! 8/70
宜蘭縣宜蘭市 Finished! 9/70
屏東縣屏東市 Finished! 10/70
屏東縣恆春鎮 Finished! 11/70
屏東縣潮州鎮 Finished! 12/70
彰化縣二林鎮 Finished! 13/70
彰化縣彰化市 Finished! 14/70
彰化縣線西鄉 Finished! 15/70
新北市三重區 Finished! 16/70
新北市土城區 Finished! 17/70
新北市新店區 Finished! 18/70
新北市新莊區 Finished! 19/70
新北市板橋區 Finished! 20/70
新北市林口區 Finished! 21/70
新北市永和區 Finished! 22/70
新北市汐止區 Finished! 23/70
新北市淡水區 Finished! 24/70
新北市石門區 Finished! 25/70
新北市萬里區 Finished! 26/70
新竹市東區 Finished! 27/70
新竹縣湖口鄉 Finished! 28/70
桃園市中壢區 Finished! 29/70
桃園市大園區 Finished! 30/70
桃園市桃園區 Finished! 31/70
桃園市觀音區 Finished! 32/70
桃園市龍潭區 Finished! 33/70
臺中市南屯區 Finished! 34/70
臺中市大里區 Finished! 35/70
臺中市沙鹿區 Finished! 36/70
臺中市西屯區 Finished! 37/70
臺中市豐原區 Finished! 38/70
臺北市中山區 Finished! 39/70
臺北市北投區 Finished! 40/70
臺北市大同區 Finished! 41/70
臺北市大安區 Finished! 42/70
臺北市松山區 Finished! 43/70
臺北市萬華區 Finished! 44/70

In [32]:
result40 = predict_201703_PM25(model40)
result50 = predict_201703_PM25(model50)
result60 = predict_201703_PM25(model60)

南投縣南投市 Finished! 1/70
南投縣埔里鎮 Finished! 2/70
南投縣竹山鎮 Finished! 3/70
嘉義市西區 Finished! 4/70
嘉義縣新港鄉 Finished! 5/70
嘉義縣朴子市 Finished! 6/70
基隆市信義區 Finished! 7/70
宜蘭縣冬山鄉 Finished! 8/70
宜蘭縣宜蘭市 Finished! 9/70
屏東縣屏東市 Finished! 10/70
屏東縣恆春鎮 Finished! 11/70
屏東縣潮州鎮 Finished! 12/70
彰化縣二林鎮 Finished! 13/70
彰化縣彰化市 Finished! 14/70
彰化縣線西鄉 Finished! 15/70
新北市三重區 Finished! 16/70
新北市土城區 Finished! 17/70
新北市新店區 Finished! 18/70
新北市新莊區 Finished! 19/70
新北市板橋區 Finished! 20/70
新北市林口區 Finished! 21/70
新北市永和區 Finished! 22/70
新北市汐止區 Finished! 23/70
新北市淡水區 Finished! 24/70
新北市石門區 Finished! 25/70
新北市萬里區 Finished! 26/70
新竹市東區 Finished! 27/70
新竹縣湖口鄉 Finished! 28/70
桃園市中壢區 Finished! 29/70
桃園市大園區 Finished! 30/70
桃園市桃園區 Finished! 31/70
桃園市觀音區 Finished! 32/70
桃園市龍潭區 Finished! 33/70
臺中市南屯區 Finished! 34/70
臺中市大里區 Finished! 35/70
臺中市沙鹿區 Finished! 36/70
臺中市西屯區 Finished! 37/70
臺中市豐原區 Finished! 38/70
臺北市中山區 Finished! 39/70
臺北市北投區 Finished! 40/70
臺北市大同區 Finished! 41/70
臺北市大安區 Finished! 42/70
臺北市松山區 Finished! 43/70
臺北市萬華區 Finished! 44/70

In [29]:
result15.to_csv('output_epoch15.csv', index=False, header=True)
result20.to_csv('output_epoch20.csv', index=False, header=True)
result30.to_csv('output_epoch30.csv', index=False, header=True)

In [33]:
result40.to_csv('output_epoch40.csv', index=False, header=True)
result50.to_csv('output_epoch50.csv', index=False, header=True)
result60.to_csv('output_epoch60.csv', index=False, header=True)